# HW1 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  1
* **Date:**  5/16/16

**HW0.0:** Prepare your bio and include it in this HW submission. Please limit to 100 words. Count the words in your bio and print the length of your bio (in terms of words) in a separate cell.

**Bio:**
I have a bachelor and master degree in computer science from MIT.  I worked for 10 years as a technical project manager at 3 different companies:  Trilogy software, Nordstrom and Chase Bank.  I am enrolled in the master of data science (MIDS) program at UC Berkeley and expect to graduate in August 2016.  I am interested in machine learning in general and specifically machine learning at scale and deep learning.  I live in Seattle, Washington.

**Bio word length:**  75 words

**HW1.0.0.** Define big data. Provide an example of a big data problem in your domain of expertise. 

**HW1.0.1.**  Bias Variance.  In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreduciable error for a test dataset T when using polynomial regression models of degree 1, 2, 3, 4, 5 are considered. How would you select a model?

Not included in this document.  Due date extended to 5/20/16

**HW1.1.** Read through the provided control script (pNaiveBayes.sh) and all of its comments. When you are comfortable with their purpose and function, respond to the remaining homework questions below.  A simple cell in the notebook with a print statmement with a "done" string will suffice here.

In [3]:
print('done')

done


**HW1.2.** Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.

The results from running the command line code './pNaiveBayes.sh 4 "assistance"' are as follows:

Total number of occurances of "assistance" is: 10

In [1]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.2.

# Import sys library to access arguments passed in when the module is called
import sys
# Import re library to manipulate regular expressions
import re
# Create a variable to store the count of the word that is passed in to the function
# Initialize this count to 0.
count = 0
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

## collect user input
# filename is the 1st argument passed in to the module and is the name of the file
# that is to be analyzed.
filename = sys.argv[1]
# findword is the 2nd argument passed in to the module and is the word whose
# occurances will be counted.  Convert the string to lowercase.
findword = sys.argv[2].lower()

# Open the file passed in to the module
with open (filename, "r") as myfile:
    # Loop through each line of the file.
    for line in myfile.readlines():
        # Create a list of words found in each line
        words = re.findall(WORD_RE, line.lower())
        # Count the number of occruances of findword in the words list.
        count += words.count(findword)

# Print the name of the word and the number of occurances of the word.  These values
# will be passed to reducer.py
print('%s %d' % (findword, count))

Overwriting mapper.py


In [2]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.2

# Import sys library to access arguments passed in when the module is called
import sys

# Create a variable to store the total number occurances of the word that was given by the user
# Initialize this total to 0.
total = 0

# Loop through each of the files that are passed into the module.
for i in range(1, len(sys.argv)):
    # Each argument that was passed in is a filename.
    filename = sys.argv[i]
    # Open the file
    with open (filename, "r") as myfile:
        # Loop through each line of the file.
        for line in myfile.readlines():
            # Split the line by spaces
            words = line.split(" ")
            # The 1st value in the line is the word that is being analyzed.
            findword = words[0]
            # The 2nd value in the line is the count of the word from each file.  Add the count from
            # this line to the total count variable.
            total += int(words[1])

# Print the word that is being analyzed as well as the total number of occurances of the word.
print('Total number of occurances of %s is: %d' % (findword, total))

Overwriting reducer.py


In [3]:
# Change the permissions for mapper.py, reducer.py and pNaiveBayes.sh so that they include
# execute permissions.
!chmod +x mapper.py
!chmod +x reducer.py
!chmod a+x pNaiveBayes.sh

In [4]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4 "assistance"

In [5]:
# Report the results from HW1.2 be examining the output file: 'enronemail_1h.txt.output' 
!cat enronemail_1h.txt.output

Total number of occurances of assistance is: 10


**HW1.3.** HW1.3. Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will classify the email messages by a single, user-specified word using the multinomial Naive Bayes Formulation. Examine the word “assistance” and report your results.
   
The results from running the command line code './pNaiveBayes.sh 4 "assistance"' are that as follows:

The program had an error rate of 40%.  It classified 40 documents incorrectly.

In [7]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.3

# Import sys library to access arguments passed in when the module is called
import sys
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

## collect user input
# filename is the 1st argument passed in to the module and is the name of the file
# that is to be analyzed.
filename = sys.argv[1]
# vocab is the 2nd argument passed in to the module and is the word that will be used
# to classify the documents.  Convert the string to lowercase.
vocab = sys.argv[2].lower()

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the word that is
# being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = [0, 0]

# Open the file that was passed in
with open (filename, "r") as myfile:
    # For each line of the file
    for line in myfile.readlines():
        # Split the line tabs and store the result in the 'items' list.
        items = line.split('\t')
        # The 1st value of the line is the ID of the document
        ID = items[0]
        # The 2nd value of the line is the true classification of the document called: TRUTH
        TRUTH = int(items[1])
        # The 3rd value of the line is the subject.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_subject = re.findall(WORD_RE, items[2].lower())
        # The 4th value of the line is the body.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_body = re.findall(WORD_RE, items[3].lower())
        # Concatenate the subject and body in to one list called 'words_all'
        words_all = words_subject + words_body
        # Update 'count_docs' by 1 for the class of this document
        count_docs[TRUTH] += 1
        # Update 'count_words' by the length of the 'words_all' list for the class of this document
        count_words[TRUTH] += len(words_all)
        # Store the number of occurances of the vocab word in the document
        vocab_terms_in_doc = words_all.count(vocab)
        # Update 'count_vocab_terms' for the class of this document
        count_vocab_terms[TRUTH] += vocab_terms_in_doc 
        # Print the ID, TRUTH and count of the vocab word that get passed to reducer.py
        print('%s\t%s\t%d' % (ID, TRUTH, vocab_terms_in_doc))

# Print the count totals for each class that get passed to reducer.py
print('count_docs\t%d\t%d' % (count_docs[0], count_docs[1]))
print('count_words\t%d\t%d' % (count_words[0], count_words[1]))
print('count_vocab_terms\t%d\t%d' % (count_vocab_terms[0], count_vocab_terms[1]))

Overwriting mapper.py


In [18]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.3

# Import sys library to access arguments passed in when the module is called
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math

# Initialize a dictionary 'all_docs' which will store the necessary information for each
# document that will be passed to the reducer.py.
all_docs = {}
# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the word that is
# being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = [0, 0]

# Loop through each of the files that are passed into the module.
for i in range(1, len(sys.argv)):
    # Each argument that was passed in is a filename.
    filename = sys.argv[i]
    # Open the file
    with open (filename, "r") as myfile:
        # For each line in the file
        for line in myfile.readlines():
            # Split the line by tabs
            elements = line.split("\t")
            # The 1st value in the line is the ID of the document
            ID = elements[0]
            # If the ID is one of the labels that reports the counts of an entire file, the
            # store that value appropriately.
            if ID[:6] == 'count_':
                # Update the 'count_docs' variable with the numbers from the file for each class
                if ID == 'count_docs':
                    count_docs[0] += int(elements[1])
                    count_docs[1] += int(elements[2])
                # Update the 'count_words' variable with the numbers from the file for each class
                elif ID == 'count_words':
                    count_words[0] += int(elements[1])
                    count_words[1] += int(elements[2])
                # Update the 'count_vocab_terms' variable with the numbers from the file for each class
                else:
                    count_vocab_terms[0] += int(elements[1])
                    count_vocab_terms[1] += int(elements[2])                 
            else:
                # Save the document in the all_docs dictionary for later use.  Use the ID as the key
                # for the dictionary.  Store the TRUTH value and the # of occurances of the vocab
                # word in a list.
                TRUTH = int(elements[1])
                vocab_terms_in_doc = int(elements[2])
                all_docs[ID] = [TRUTH, vocab_terms_in_doc]

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Calculate the total number of documents by adding the total counts from each class.
total_docs = sum(count_docs)
# Initialize the 'prior' varialbe to store the prior probability for each class.
prior = [0, 0]
# Initialize the 'condprob' varialbe to store the conditional probilitity of the vocab word for
# each class.
condprob = [0, 0]

# Train the MultinomialNB classifer using the algorithm in the book: An Introduction to Information Retrieval
# By Christopher D. Manning, Prabhakar Raghavan & Hinrich Schutzepage page 260, Figure 13.2.
# For each of the classes:
# 1. Calculate the prior probability (prior) by dividing the total # of documents in that class by the total # of documents.
# 2. Calculate the conditional probability (condprob) by dividing the total # of terms in that class by the 
# total number of terms in all document in that class
# Do not use smoothing.
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    condprob[cls] = count_vocab_terms[cls] / float(count_words[cls])

# Initialize variables to track the total correct predictions and the total predictions
count_correct = 0
count_total = 0
# Apply the MultinomialNB classifier to each document and make a prediction for each.
# For each document:
# 1. Initialize a score of 0 for each class
# 2. Store the true value for that document in the 'TRUTH' variable
# 3. Store the # of times the vocab term occurs in the document
# 4. For each class: calculate the score as a sum of the log of the prior probability + 
# (# of times the vocab term occurs in the document) * log of the conditional probability for the
# vocab term.
# 5. Make a prediction:  Choose the class with the highest score.
# 6. Print the values to the output file:  ID, true value, prediction
for key, value in all_docs.iteritems():
    score = [0, 0]
    TRUTH = value[0]
    vocab_terms_in_doc = value[1]
    for cls in classes:
        score[cls] = math.log(prior[cls])
        score[cls] += vocab_terms_in_doc * math.log(condprob[cls])
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (key, TRUTH, prediction))
    # Update the totals used for accuracy
    count_total += 1
    if TRUTH == prediction:
        count_correct += 1
print('Training Error: %f' % ((count_total - count_correct) / float(count_total)))

Overwriting reducer.py


In [19]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4 "assistance"

In [20]:
# Report the results from HW1.3 be examining the output file: 'enronemail_1h.txt.output' 
!cat enronemail_1h.txt.output

0010.2003-12-18.GP	1	0
0010.2001-06-28.SA_and_HP	1	1
0001.2000-01-17.beck	0	0
0018.1999-12-14.kaminski	0	0
0005.1999-12-12.kaminski	0	1
0011.2001-06-29.SA_and_HP	1	0
0008.2004-08-01.BG	1	0
0009.1999-12-14.farmer	0	0
0017.2003-12-18.GP	1	0
0011.2001-06-28.SA_and_HP	1	1
0015.2001-07-05.SA_and_HP	1	0
0015.2001-02-12.kitchen	0	0
0009.2001-06-26.SA_and_HP	1	0
0017.1999-12-14.kaminski	0	0
0012.2000-01-17.beck	0	0
0003.2000-01-17.beck	0	0
0004.2001-06-12.SA_and_HP	1	0
0008.2001-06-12.SA_and_HP	1	0
0007.2001-02-09.kitchen	0	0
0016.2004-08-01.BG	1	0
0015.2000-06-09.lokay	0	0
0005.1999-12-14.farmer	0	0
0016.1999-12-15.farmer	0	0
0013.2004-08-01.BG	1	1
0005.2003-12-18.GP	1	0
0012.2001-02-09.kitchen	0	0
0003.2001-02-08.kitchen	0	0
0009.2001-02-09.kitchen	0	0
0006.2001-02-08.kitchen	0	0
0014.2003-12-19.GP	1	0
0010.1999-12-14.farmer	0	0
0010.2004-08-01.BG	1	0
0014.1999-12-14.kaminski	0	0
0006.1999-12-13.kaminski	0	0
0011.1999-12-14.farmer	0	0
0013.1999-12-14.kaminski	0	0
0001.2001-02-07.kitchen	0	0


**HW1.4.** Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will classify the email messages by a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results (accuracy).

The results from running the command line code './pNaiveBayes.sh 4 "assistance valium enlargementWithATypo"' are that as follows:

* The program had an accuracy of 60%. It classified 60 documents correctly.
* The program had an error rate of 40%. It classified 40 documents incorrectly.

In [22]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.4

# Import pring function from python 3
from __future__ import print_function
# Import sys library to access arguments passed in when the module is called
import sys
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

## collect user input
# filename is the 1st argument passed in to the module and is the name of the file
# that is to be analyzed.
filename = sys.argv[1]
# vocab is the 2nd argument passed in to the module and is the list of words that will be used
# to classify the documents.  Convert the string to lowercase and create a list called vocab
# that stores each word passed in.
vocab = re.split(" ",sys.argv[2].lower())

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the words in the 
# vocab that is being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
for term in vocab:
    count_vocab_terms[term] = [0, 0]

# Open the file that was passed in
with open (filename, "r") as myfile:
    # For each line of the file
    for line in myfile.readlines():
        # Split the line tabs and store the result in the 'items' list.
        items = line.split('\t')
        # The 1st value of the line is the ID of the document
        ID = items[0]
        # The 2nd value of the line is the true classification of the document called: TRUTH
        TRUTH = int(items[1])
        # The 3rd value of the line is the subject.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_subject = re.findall(WORD_RE, items[2].lower())
        # The 4th value of the line is the body.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_body = re.findall(WORD_RE, items[3].lower())
        # Concatenate the subject and body in to one list called 'words_all'
        words_all = words_subject + words_body
        # Update 'count_docs' by 1 for the class of this document
        count_docs[TRUTH] += 1
        # Update 'count_words' by the length of the 'words_all' list for the class of this document
        count_words[TRUTH] += len(words_all)
        # Print the ID and true classification to send reducer.py
        print('%s\t%s' % (ID, TRUTH),end="")
        # For each of the words in the vocabulary print out the 'term' and the '# of occurances
        # of the term' for reducer.py
        for term in vocab:
            # Store the number of occurances of the vocab word in the document
            count_term = words_all.count(term)
            # Update 'count_vocab_terms' for the class of this document
            count_vocab_terms[term][TRUTH] += count_term
            print('\t%s\t%d' % (term, count_term),end="")
        print()

# Print the count totals for each class that get passed to reducer.py
print('count_docs\t%d\t%d' % (count_docs[0], count_docs[1]))
print('count_words\t%d\t%d' % (count_words[0], count_words[1]))
for term in vocab:
    print('count_vocab_term\t%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))

Overwriting mapper.py


In [23]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.4

# Import sys library to access arguments passed in when the module is called
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math

# Initialize a dictionary 'all_docs' which will store the necessary information for each
# document that will be passed to the reducer.py.
all_docs = {}
# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the each of the 
# words in the vocab that is being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
vocab = []
# Keep track of the first file in order to just store once the vocab that is being analyzed
FIRST_FILE = True

# Loop through each of the files that are passed into the module.
for i in range(1, len(sys.argv)):
    # Each argument that was passed in is a filename.
    filename = sys.argv[i]
    # Open the file
    with open (filename, "r") as myfile:
        # For each line in the file
        for line in myfile.readlines():
            # Split the line by tabs
            elements = line.split("\t")
            # The 1st value in the line is the ID of the document
            ID = elements[0]
            # If the ID is one of the labels that reports the counts of an entire file, the
            # store that value appropriately.
            if ID[:6] == 'count_':
                # Update the 'count_docs' variable with the numbers from the file for each class
                if ID == 'count_docs':
                    count_docs[0] += int(elements[1])
                    count_docs[1] += int(elements[2])
                # Update the 'count_words' variable with the numbers from the file for each class
                elif ID == 'count_words':
                    count_words[0] += int(elements[1])
                    count_words[1] += int(elements[2])
                # Update the 'count_vocab_terms' variable with the numbers from the file for each class
                else:
                    term = elements[1]
                    if term not in count_vocab_terms:
                        count_vocab_terms[term] = [int(elements[2]), int(elements[3])]
                    else:
                        count_vocab_terms[term][0] += int(elements[2])
                        count_vocab_terms[term][1] += int(elements[3])
                    # if it's the first file, save the vocab
                    if FIRST_FILE:
                        vocab.append(term)
            else:
                # Save the document in the all_docs dictionary for later use.  Use the ID as the key
                # for the dictionary.  Store the TRUTH value and a dictionary consisting of the each
                # term in the vocab followed by # of occurances of that term
                TRUTH = int(elements[1])
                terms = {}
                for j in range(2, len(elements),2):
                    term = elements[j]
                    terms[term] = int(elements[j+1])
                all_docs[ID] = [TRUTH, terms]
    # Update the FIRST_FILE variable after the first file is read.
    FIRST_FILE = False

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Calculate the total number of documents by adding the total counts from each class.
total_docs = sum(count_docs)
# Initialize the 'prior' varialbe to store the prior probability for each class.
prior = [0, 0]
# Initialize the 'condprob' varialbe to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}
for term in vocab:
    condprob[term] = [0, 0]

# Train the MultinomialNB classifer using the algorithm in the book: An Introduction to Information Retrieval
# By Christopher D. Manning, Prabhakar Raghavan & Hinrich Schutzepage page 260, Figure 13.2.
# For each of the classes:
# 1. Calculate the prior probability (prior) by dividing the total # of documents in that class by the total # of documents.
# 2. Calculate the conditional probability (condprob) for each term in the vocab by dividing the
# total # of that term in that class by the total number of terms in all documents in that class
# Do not use smoothing.
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    for term in vocab:
        condprob[term][cls] = count_vocab_terms[term][cls] / float(count_words[cls])

# Initialize variables to track the total correct predictions and the total predictions
count_correct = 0
count_total = 0
# Apply the MultinomialNB classifier to each document and make a prediction for each.
# For each document:
# 1. Initialize a score of 0 for each class
# 2. Store the true value for that document in the 'TRUTH' variable
# 3. Store the # of times the vocab term occurs in the document
# 4. For each class: calculate the score as a sum of the log of the prior probability + 
# for each term in the vocab: (# of times the vocab term occurs in the document) * log of the
# conditional probability for the vocab term.
# 5. Make a prediction:  Choose the class with the highest score.
# 6. Print the values to the output file:  ID, true value, prediction
for ID, value in all_docs.iteritems():
    score = [0, 0]
    TRUTH = value[0]
    vocab_terms_in_doc = value[1]
    for cls in classes:
        score[cls] = math.log(prior[cls])
        for term, count in vocab_terms_in_doc.iteritems():
            if condprob[term][cls] > 0.0:
                score[cls] += (count * math.log(condprob[term][cls]))
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (ID, TRUTH, prediction))
    # Update the totals used for accuracy
    count_total += 1
    if TRUTH == prediction:
        count_correct += 1
print('Accuracy: %f' % (count_correct / float(count_total)))
print('Training Error: %f' % ((count_total - count_correct) / float(count_total)))

Overwriting reducer.py


In [24]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4 "assistance valium enlargementWithATypo"

In [25]:
# Report the results from HW1.4 be examining the output file: 'enronemail_1h.txt.output' 
!cat enronemail_1h.txt.output

0010.2003-12-18.GP	1	0
0010.2001-06-28.SA_and_HP	1	1
0001.2000-01-17.beck	0	0
0018.1999-12-14.kaminski	0	0
0005.1999-12-12.kaminski	0	1
0011.2001-06-29.SA_and_HP	1	0
0008.2004-08-01.BG	1	0
0009.1999-12-14.farmer	0	0
0017.2003-12-18.GP	1	0
0011.2001-06-28.SA_and_HP	1	1
0015.2001-07-05.SA_and_HP	1	0
0015.2001-02-12.kitchen	0	0
0009.2001-06-26.SA_and_HP	1	0
0017.1999-12-14.kaminski	0	0
0012.2000-01-17.beck	0	0
0003.2000-01-17.beck	0	0
0004.2001-06-12.SA_and_HP	1	0
0008.2001-06-12.SA_and_HP	1	0
0007.2001-02-09.kitchen	0	0
0016.2004-08-01.BG	1	0
0015.2000-06-09.lokay	0	0
0005.1999-12-14.farmer	0	0
0016.1999-12-15.farmer	0	0
0013.2004-08-01.BG	1	1
0005.2003-12-18.GP	1	0
0012.2001-02-09.kitchen	0	0
0003.2001-02-08.kitchen	0	0
0009.2001-02-09.kitchen	0	0
0006.2001-02-08.kitchen	0	0
0014.2003-12-19.GP	1	0
0010.1999-12-14.farmer	0	0
0010.2004-08-01.BG	1	0
0014.1999-12-14.kaminski	0	0
0006.1999-12-13.kaminski	0	0
0011.1999-12-14.farmer	0	0
0013.1999-12-14.kaminski	0	0
0001.2001-02-07.kitchen	0	0


**HW1.5.** Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will classify the email messages by all words present.
   
The results from running the command line code './pNaiveBayes.sh 4' are that as follows:

The program had an error rate of 0%.  It classified 0 documents incorrectly.

In [1]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.5

# Import pring function from python 3
from __future__ import print_function
# Import sys library to access arguments passed in when the module is called
import sys
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

## collect user input
# filename is the 1st argument passed in to the module and is the name of the file
# that is to be analyzed.
filename = sys.argv[1]

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the words in the 
# vocab that is being used for classification in each of the classes.
# Variable vocab:  stores the unique terms used in this file
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
vocab = []

# Open the file that was passed in
with open (filename, "r") as myfile:
    # For each line of the file
    for line in myfile.readlines():
        # Split the line tabs and store the result in the 'items' list.
        items = line.split('\t')
        # The 1st value of the line is the ID of the document
        ID = items[0]
        # The 2nd value of the line is the true classification of the document called: TRUTH
        TRUTH = int(items[1])
        # The 3rd value of the line is the subject.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_subject = re.findall(WORD_RE, items[2].lower())
        # The 4th value of the line is the body.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_body = re.findall(WORD_RE, items[3].lower())
        # Concatenate the subject and body in to one list called 'words_all'
        words_all = words_subject + words_body
        # Update 'count_docs' by 1 for the class of this document
        count_docs[TRUTH] += 1
        # Update 'count_words' by the length of the 'words_all' list for the class of this document
        count_words[TRUTH] += len(words_all)
        # Create a variable to store the unique words from this line.
        vocab_line = []
        # Loop through each of the terms in the words of this line
        for term in words_all:
            # Update the vocab for this file
            if term not in vocab:
                vocab.append(term)
            # Update the vocab for this line
            if term not in vocab_line:
                vocab_line.append(term)
            # Initialize a list to store the count of this term
            if term not in count_vocab_terms:
                count_vocab_terms[term] = [0,0]
        # Print the ID and true classification to send reducer.py
        print('%s\t%s' % (ID, TRUTH),end="")
        # For each of the words in the vocabulary print out the 'term' and the '# of occurances
        # of the term' for reducer.py
        for term in vocab_line:
            # Store the number of occurances of the vocab word in the document
            count_term = words_all.count(term)
            # Update 'count_vocab_terms' for the class of this document
            count_vocab_terms[term][TRUTH] += count_term
            print('\t%s\t%d' % (term, count_term),end="")
        print()

# Print the count totals for each class that get passed to reducer.py
print('count_docs\t%d\t%d' % (count_docs[0], count_docs[1]))
print('count_words\t%d\t%d' % (count_words[0], count_words[1]))
for term in vocab:
    print('count_vocab_term\t%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))

Overwriting mapper.py


In [2]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.5

# Import sys library to access arguments passed in when the module is called
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math

# Initialize a dictionary 'all_docs' which will store the necessary information for each
# document that will be passed to the reducer.py.
all_docs = {}
# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the each of the 
# words in the vocab that is being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}

# Loop through each of the files that are passed into the module.
for i in range(1, len(sys.argv)):
    # Each argument that was passed in is a filename.
    filename = sys.argv[i]
    # Open the file
    with open (filename, "r") as myfile:
        # For each line in the file
        for line in myfile.readlines():
            # Split the line by tabs
            elements = line.split("\t")
            # The 1st value in the line is the ID of the document
            ID = elements[0]
            # If the ID is one of the labels that reports the counts of an entire file, the
            # store that value appropriately.
            if ID[:6] == 'count_':
                # Update the 'count_docs' variable with the numbers from the file for each class
                if ID == 'count_docs':
                    count_docs[0] += int(elements[1])
                    count_docs[1] += int(elements[2])
                # Update the 'count_words' variable with the numbers from the file for each class
                elif ID == 'count_words':
                    count_words[0] += int(elements[1])
                    count_words[1] += int(elements[2])
                # Update the 'count_vocab_terms' variable with the numbers from the file for each class
                else:
                    term = elements[1]
                    if term not in count_vocab_terms:
                        count_vocab_terms[term] = [int(elements[2]), int(elements[3])]
                    else:
                        count_vocab_terms[term][0] += int(elements[2])
                        count_vocab_terms[term][1] += int(elements[3])
            else:
                # Save the document in the all_docs dictionary for later use.  Use the ID as the key
                # for the dictionary.  Store the TRUTH value and a dictionary consisting of each
                # term in the vocab followed by # of occurances of that term
                TRUTH = int(elements[1])
                terms = {}
                for j in range(2, len(elements),2):
                    term = elements[j]
                    terms[term] = int(elements[j+1])
                all_docs[ID] = [TRUTH, terms]


# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Calculate the total number of documents by adding the total counts from each class.
total_docs = sum(count_docs)
# Initialize the 'prior' varialbe to store the prior probability for each class.
prior = [0, 0]
# Initialize the 'condprob' varialbe to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}
for term in count_vocab_terms:
    condprob[term] = [0, 0]

# Train the MultinomialNB classifer using the algorithm in the book: An Introduction to Information Retrieval
# By Christopher D. Manning, Prabhakar Raghavan & Hinrich Schutzepage page 260, Figure 13.2.
# For each of the classes:
# 1. Calculate the prior probability (prior) by dividing the total # of documents in that class by the total # of documents.
# 2. Calculate the conditional probability (condprob) for each term in the vocab by dividing the
# total # of that term in that class by the total number of terms in all documents in that class
# Use smoothing by adding 1.0 to the numerator and the denominator in the condprob equation.
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    for term in count_vocab_terms:
        condprob[term][cls] = (count_vocab_terms[term][cls]+1.0) / (float(count_words[cls])+1.0)

# Initialize variables to track the total correct predictions and the total predictions
count_correct = 0
count_total = 0
# Apply the MultinomialNB classifier to each document and make a prediction for each.
# For each document:
# 1. Initialize a score of 0 for each class
# 2. Store the true value for that document in the 'TRUTH' variable
# 3. Store the # of times the vocab term occurs in the document
# 4. For each class: calculate the score as a sum of the log of the prior probability + 
# for each term in the vocab: (# of times the vocab term occurs in the document) * log of the
# conditional probability for the vocab term.
# 5. Make a prediction:  Choose the class with the highest score.
# 6. Print the values to the output file:  ID, true value, prediction
for ID, value in all_docs.iteritems():
    score = [0, 0]
    TRUTH = value[0]
    vocab_terms_in_doc = value[1]
    for cls in classes:
        score[cls] = math.log(prior[cls])
        for term, count in vocab_terms_in_doc.iteritems():
            if condprob[term][cls] > 0.0:
                score[cls] += (count * math.log(condprob[term][cls]))
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (ID, TRUTH, prediction))
    # Update the totals used for accuracy
    count_total += 1
    if TRUTH == prediction:
        count_correct += 1
print('Accuracy: %f' % (count_correct / float(count_total)))
print('Training Error: %f' % ((count_total - count_correct) / float(count_total)))

Overwriting reducer.py


In [3]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4

In [4]:
# Report the results from HW1.5 be examining the output file: 'enronemail_1h.txt.output' 
!cat enronemail_1h.txt.output

0010.2003-12-18.GP	1	1
0010.2001-06-28.SA_and_HP	1	1
0001.2000-01-17.beck	0	0
0018.1999-12-14.kaminski	0	0
0005.1999-12-12.kaminski	0	0
0011.2001-06-29.SA_and_HP	1	1
0008.2004-08-01.BG	1	1
0009.1999-12-14.farmer	0	0
0017.2003-12-18.GP	1	1
0011.2001-06-28.SA_and_HP	1	1
0015.2001-07-05.SA_and_HP	1	1
0015.2001-02-12.kitchen	0	0
0009.2001-06-26.SA_and_HP	1	1
0017.1999-12-14.kaminski	0	0
0012.2000-01-17.beck	0	0
0003.2000-01-17.beck	0	0
0004.2001-06-12.SA_and_HP	1	1
0008.2001-06-12.SA_and_HP	1	1
0007.2001-02-09.kitchen	0	0
0016.2004-08-01.BG	1	1
0015.2000-06-09.lokay	0	0
0005.1999-12-14.farmer	0	0
0016.1999-12-15.farmer	0	0
0013.2004-08-01.BG	1	1
0005.2003-12-18.GP	1	1
0012.2001-02-09.kitchen	0	0
0003.2001-02-08.kitchen	0	0
0009.2001-02-09.kitchen	0	0
0006.2001-02-08.kitchen	0	0
0014.2003-12-19.GP	1	1
0010.1999-12-14.farmer	0	0
0010.2004-08-01.BG	1	1
0014.1999-12-14.kaminski	0	0
0006.1999-12-13.kaminski	0	0
0011.1999-12-14.farmer	0	0
0013.1999-12-14.kaminski	0	0
0001.2001-02-07.kitchen	0	0


**HW1.6.** Benchmark your code with the Python SciKit-Learn implementation of multinomial Naive Bayes.

In this exercise, please complete the following:

1. Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW1.5 and report the Training error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)
2. Run the Bernoulli Naive Bayes algorithm from SciKit-Learn (using default settings) over the same training data used in HW1.5 and report the Training error 
3. Run the Multinomial Naive Bayes algorithm you developed for HW1.5 over the same data used HW1.5 and report the Training error 
4. Please prepare a table to present your results
5. Explain/justify any differences in terms of training error rates over the dataset in HW1.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn (Hint: smoothing, which we will discuss in next lecture)
6. Discuss the performance differences in terms of training error rates over the dataset in HW1.5 between the  Multinomial Naive Bayes implementation in SciKit-Learn with the  Bernoulli Naive Bayes implementation in SciKit-Learn
7. What is the accuracy of the your SciKit-Learn model on the training data for a Naive Bayes model with smoothing and without smoothing using all the vocabulary? Please comment on what you see? Explain!

In [5]:
# Import SK-learn libraries for learning.
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
# Import SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Transform the training data into a format that the sklearn libraries can use.
# train_ids:  stores the IDs from the training data.
train_ids = []
# train_data:  stores the text from each document
train_data = []
# train_labels:  stores the true labels for each document
train_labels = []
# set filename to the file that is being analyzed
filename = "enronemail_1h.txt"
# Open the file
with open (filename, "r") as myfile:
    # Loop through each line in the file
    for line in myfile.readlines():
        # Split the lines by tab
        items = line.split('\t')
        # Add each ID to the 'train_ids' list
        train_ids.append(items[0])
        # Add each label to the 'train_labels' list
        train_labels.append(int(items[1]))
        # The 3rd value of the line is the subject.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_subject = re.findall(WORD_RE, items[2].lower())
        # The 4th value of the line is the body.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_body = re.findall(WORD_RE, items[3].lower())
        # Concatenate the subject and body in to one list called 'words_all'
        words_all = words_subject + words_body
        # Create one string containing all of the words in the document separated by spaces
        # and then add that string to the 'train_data' list
        train_data.append(' '.join(words_all))

# Create a CountVectorizer object that will transform the text data into a feature vector
cv = CountVectorizer()
# Transform the train_data into a feature vector
train_fv = cv.fit_transform(train_data)

# Create a MultinomialNB model using the default parameters
multiNB = MultinomialNB()
# Fit the model with the training data
multiNB.fit(train_fv, train_labels)
# Create predictions from the model using the training data
predict = multiNB.predict(train_fv)

# Initialize variables to track the total incorrect predictions and the total predictions
count_total = 0
count_incorrect = 0
# Calculate the training error by dividing the count_incorrect by the total count
for i in range(len(predict)):
    if predict[i] != train_labels[i]:
        count_incorrect += 1
    count_total += 1
print('Training Error: %f' % (count_incorrect / float(count_total)))

# Create a MultinomialNB model
BernoulliNB = BernoulliNB()
# Fit the model with the training data
BernoulliNB.fit(train_fv, train_labels)
# Create predictions from the model using the training data
predict = BernoulliNB.predict(train_fv)

# Initialize variables to track the total incorrect predictions and the total predictions
count_total = 0
count_incorrect = 0
# Calculate the training error by dividing the count_incorrect by the total count
for i in range(len(predict)):
    if predict[i] != train_labels[i]:
        count_incorrect += 1
    count_total += 1
print('Training Error: %f' % (count_incorrect / float(count_total)))

Training Error: 0.000000
Training Error: 0.160000


In [13]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.5_OLD

import sys
import re

# Takes a list of words and returns a string separated by spaces of the
# unique words in the list
def findUniqueWords(word_list):
    unique = []
    for w in word_list:
        if w not in unique:
            unique.append(w)
    return ' '.join(unique)

WORD_RE = re.compile(r"[\w']+")

## collect user input
filename = sys.argv[1]
#findwords = re.split(" ",sys.argv[2].lower())
findword = sys.argv[2].lower()

with open (filename, "r") as myfile:
    for line in myfile.readlines():
        items = line.split('\t')
        ID = items[0]
        TRUTH = items[1]
        # Create a list of words found in each line
        words_subject = re.findall(WORD_RE, items[2].lower())
        words_body = re.findall(WORD_RE, items[3].lower())
        words_all = words_subject + words_body
        count_total_words = len(words_all)
        all_text = ' '.join(words_all)
        vocab = findUniqueWords(words_all)
        print('%s\t%s\t%d\t%s\t%s' % (ID, TRUTH, count_total_words, vocab, all_text))


Overwriting mapper.py


In [3]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.5_OLD
import sys

def updateVocab(V, word_list):
    for w in word_list:
        if w not in V:
            V.add(w)
    return V

def countTokensOfTerm(word_list, term):
    return word_list.count(term)

all_docs = {}
all_vocab = set()
count_docs = [0, 0]
count_words = [0, 0]
all_text = [[], []]

# Loop through the files once to create the aggregate counts
for i in range(1, len(sys.argv)):
    filename = sys.argv[i]
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            elements = line.split("\t")
            ID = elements[0]
            TRUTH = int(elements[1])
            count_docs[TRUTH] += 1
            count_words[TRUTH] += int(elements[2])
            vocab = elements[3].split(' ')
            all_vocab = updateVocab(all_vocab, vocab)
            text = elements[4].split(' ')
            all_text[TRUTH] += text
            # save the document for later use
            all_docs[ID] = [TRUTH, text]

total_docs = sum(count_docs)
prior = [0, 0]
condprob = [{}, {}]
classes = [0, 1]

# Train MultinomialNB
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    for term in vocab:
        count_term = countTokensOfTerm(all_text[cls], term)
        condprob[cls][term] = (count_term + 1.0) / (count_words[cls] + 1.0)

# Apply MultinomialNB
for key, value in all_docs:
    score = [0, 0]
    TRUTH = value[0]
    for cls in classes:
        score[cls] = log(prior[cls])
        text = value[1]
        for term in text:
            score[cls] += log(condprob[cls][term])
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (key, TRUTH, prediction))

Overwriting reducer.py


In [12]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 99 "assistance"

Traceback (most recent call last):
  File "./reducer.py", line 52, in <module>
    for key, value in all_docs:
ValueError: too many values to unpack


In [16]:
%%writefile mapper2.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[2]
findword = sys.argv[1]
with open (filename, "r") as myfile:
    #Please insert your code
    for line in myfile.readlines():
        # Create a list of words found in each line
        words = re.findall(WORD_RE, line)
        count += words.count(findword)
print count

Overwriting mapper.py


In [2]:
!chmod a+x mapper.py

#Reduce

In [22]:
%%writefile reducer2.py
#!/usr/bin/python
import sys
sum = 0
for line in sys.stdin:
    #Please insert your code
    sum += int(line)
print sum

Overwriting reducer.py


In [4]:
!chmod a+x reducer.py

# Write script to file

In [5]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Writing pGrepCount.sh


#Run the file

In [6]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [15]:
!./pGrepCount.sh License.txt COPYRIGHT 4k

found [11] [COPYRIGHT] in the file [License.txt]
